In [ ]:
import h5py, torch, os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# --- ١. ڕێکخستنی بنەڕەتی ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PHONEMES = ['BLANK', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH', 'SIL']
PHONEME_MAP = {p: i for i, p in enumerate(PHONEMES)}

TRAIN_FILES = [
    '/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final/t15.2025.03.14/data_train.hdf5',
    '/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final/t15.2023.08.11/data_train.hdf5',
    '/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final/t15.2023.11.19/data_train.hdf5'
]

# --- ٢. دروستکردنی Dataset (بەشی چاککراو) ---
class BrainDataset(Dataset):
    def __init__(self, file_paths):
        self.data = []
        for path in file_paths:
            if not os.path.exists(path): continue
            with h5py.File(path, 'r') as hf:
                for k in hf.keys():
                    feat = hf[k]['input_features'][()]
                    
                    # لێرەدا کێشەکە هەبوو، ئێستا چاککراوە:
                    raw_text = hf[k]['transcription'][()]
                    
                    # پشکنین: ئەگەر بایت بوو دیکۆدی بکە، ئەگەر نا ڕاستەوخۆ بەکاریبهێنە
                    if isinstance(raw_text, bytes):
                        p_text = raw_text.decode('utf-8').split()
                    elif isinstance(raw_text, np.ndarray):
                        # ئەگەر ئەڕەی بوو، دەیکەین بە تێکست
                        p_text = " ".join([x.decode('utf-8') if isinstance(x, bytes) else str(x) for x in raw_text]).split()
                    else:
                        p_text = str(raw_text).split()
                        
                    target = [PHONEME_MAP.get(p, 41) for p in p_text]
                    self.data.append((feat, target))

    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x).float(), torch.tensor(y).long()

# --- ٣. مۆدێل ---
class BrainCTCModel(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=256):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=3, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, len(PHONEMES))
    def forward(self, x):
        x, _ = self.lstm(x)
        return self.fc(x).log_softmax(2)

# --- ٤. مەشقپێکردن ---
def train():
    dataset = BrainDataset(TRAIN_FILES)
    model = BrainCTCModel().to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    criterion = nn.CTCLoss(blank=0, zero_infinity=True)

    print(f"🚀 دەستکردن بە مەشق لەسەر {len(dataset)} نموونە...")
    model.train()
    for epoch in range(10): 
        epoch_loss = 0
        for x, y in dataset:
            x, y = x.unsqueeze(0).to(DEVICE), y.unsqueeze(0).to(DEVICE)
            optimizer.zero_grad()
            log_probs = model(x).permute(1, 0, 2)
            input_len = torch.tensor([log_probs.size(0)], dtype=torch.long)
            target_len = torch.tensor([y.size(1)], dtype=torch.long)
            loss = criterion(log_probs, y, input_len, target_len)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1} | Loss: {epoch_loss/len(dataset):.4f}")
    return model

# --- ٥. پێشبینی ---
def submit(model):
    model.eval()
    results = []
    TEST_DIR = Path('/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final')
    test_files = sorted(list(TEST_DIR.rglob("data_test.hdf5")))
    with torch.no_grad():
        for p in tqdm(test_files):
            with h5py.File(p, "r") as hf:
                trials = sorted(hf.keys(), key=lambda k: int(k.split('_')[1]) if '_' in k else 0)
                for t in trials:
                    x = torch.from_numpy(hf[t]["input_features"][()]).float().unsqueeze(0).to(DEVICE)
                    logits = model(x)
                    indices = torch.argmax(logits[0], dim=-1).unique_consecutive()
                    pred = [PHONEMES[i] for i in indices if i != 0 and i < len(PHONEMES)]
                    results.append(" ".join(pred) if pred else "SIL")
    pd.DataFrame({"id": range(len(results)), "text": results}).to_csv("submission.csv", index=False)
    print("🏁 تەواو! فایلەکە ئامادەیە.")

trained_model = train()
submit(trained_model)